In [ ]:

import os
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F

# (try to) use a GPU for computation?
use_cuda=True
if use_cuda and torch.cuda.is_available():
  mydevice=torch.device('cuda')
else:
  mydevice=torch.device('cpu')

# training .npy files base dir
train_dir='../input/g2net-gravitational-wave-detection/train'
# location to store preprocessed file
cache_dir='/kaggle/temp/preprocessed/'
train_file='../input/g2net-gravitational-wave-detection/training_labels.csv'


We use the Karhunen Loeve transform, bordered autocorrelation method [Maccone, 2012](https://www.researchgate.net/publication/301171057_A_simple_introduction_to_the_KLT_and_BAM-KLT) to convert the time series data to be fed into a classifier.

In [ ]:
# KLT BAM method
def klt_bam(infile, K, M):
  # infile: input .npy file of time series
  # K: how many largest eigenvalues ?
  # M: max window size (lags) for correlation, must be > K (e.g. M=256+K)
  # output: K eigenvalues of largest magnitude 
  # input size: ndim x N
  # output size: M x K x ndim (removing zeros at beginning)
  a=np.load(infile)
  # get dimensions
  ndim,N=a.shape

  # normalize 
  for chan in range(ndim):
    a[chan]=a[chan]/np.linalg.norm(a[chan])

  assert(M>=K)

  # storage 
  eigs=np.zeros((ndim,M,K))

  # iterate over each channel
  for chan in range(ndim):
     x=a[chan]
     # find covariance
     # loop over window size
     for m in range(1,M+1):
       R=np.zeros((m,m))
       # first row
       r=np.zeros((m))
       for ci in range(m):
          r[ci]=np.dot(x[ci:N],x[0:N-ci])/(N-ci)

       # fill matrix
       for ci in range(m):
         R[ci,ci:m]=r[0:m-ci]
         R[ci+1:m,ci]=r[1:m-ci].transpose()

       # find eigenvalues (in ascending order)
       d=np.flip(np.linalg.eigvalsh(R))
       
       # select K largest
       if K>m:
         eigs[chan,m-1,K-m:K]=d[:m]
       else:
         eigs[chan,m-1,:]=d[:K]

  # calculate difference along the lag axis
  eigs=np.diff(eigs,axis=1)
  # take log(abs())
  eigs=np.log(np.abs(eigs)+1e-9)
  # select lags K to M
  eigs=eigs[:,-(M-K):,:]
  # make data zero mean unit variance
  eigs=(eigs-np.mean(eigs))/(np.sqrt(np.var(eigs)))

  return eigs


We use a 1D CNN classifier that reads in the rows .and the columns of the KLT-BAM output

In [ ]:
# 1D CNN model (both rows/columns are fed separately)
class Class1D(nn.Module):
   def __init__(self,M=256,K=12,channels=3):
     super(Class1D,self).__init__()
     self.K=K
     self.M=M
     self.channels=channels
     self.xconv0=nn.Conv1d(channels,8,3,stride=2,padding=1)
     self.xconv1=nn.Conv1d(8,12,3,stride=2,padding=1)
     self.xconv2=nn.Conv1d(12,24,3,stride=2,padding=1)
     self.xconv3=nn.Conv1d(24,48,3,stride=2,padding=1)
     self.xconv4=nn.Conv1d(48,96,3,stride=2,padding=1)
     self.yconv0=nn.Conv1d(channels,8,3,stride=2,padding=1)
     self.yconv1=nn.Conv1d(8,12,3,stride=2,padding=1)
     self.yconv2=nn.Conv1d(12,24,3,stride=2,padding=1)
     self.yconv3=nn.Conv1d(24,48,3,stride=2,padding=1)
     self.yconv4=nn.Conv1d(48,96,3,stride=2,padding=1)
     self.yconv5=nn.Conv1d(96,120,3,stride=2,padding=1)
     self.xpool0=nn.AvgPool1d(2,stride=2)
     self.xpool1=nn.AvgPool1d(2,stride=2)
     self.ypool0=nn.AvgPool1d(2,stride=2)
     self.ypool1=nn.AvgPool1d(2,stride=2)

     self.fc1=nn.Linear(96*24*2,2400)
     self.fc2=nn.Linear(2400,120)
     self.fc3=nn.Linear(120,1)


   def forward(self,E):
     # E: batch,channel,M,K
     x=torch.flatten(E,start_dim=2,end_dim=3)
     x=self.xpool0(F.elu(self.xconv0(x)))
     x=self.xpool1(F.elu(self.xconv1(x)))
     x=(F.elu(self.xconv2(x)))
     x=(F.elu(self.xconv3(x)))
     x=(F.elu(self.xconv4(x)))
     y=torch.flatten(torch.transpose(E,2,3),start_dim=2,end_dim=3)
     y=self.ypool0(F.elu(self.yconv0(y)))
     y=self.ypool1(F.elu(self.yconv1(y)))
     y=(F.elu(self.yconv2(y)))
     y=(F.elu(self.yconv3(y)))
     y=(F.elu(self.yconv4(y)))
     x=torch.flatten(x,start_dim=1)
     y=torch.flatten(y,start_dim=1)
     z=torch.cat((x,y),1)
     z=F.elu(self.fc1(z))
     z=F.elu(self.fc2(z))
     z=torch.sigmoid(self.fc3(z))
     return z

The training loop follows below, it randomly selects a minibatch of time series data, calculates the KLT for that minibatch and feeds it into the CNN classifier. It also saves the pre-processed data in a cache, to avoid calculating the KLT again if that time series is used again.

In [ ]:
# how many epochs
Nepoch=800
batchsize=40
# how many largest eigenvalues
K=12
# max window size for correlation (excluding K)
M=256

net=Class1D(M,K).to(mydevice)

load_model=False
save_model=True
# update from a saved model 
if load_model:
  checkpoint=torch.load('./net.model',map_location=mydevice)
  net.load_state_dict(checkpoint['model_state_dict'])
  net.train() # initialize for training (BN,dropout)

# get available file size
train_df=pd.read_csv(train_file)
nfiles,ncols=train_df.shape
assert(ncols==2)
del train_df


In [ ]:
import torch.optim as optim
criterion=torch.nn.BCELoss()
optimizer=optim.Adam(net.parameters(), lr=0.0001)


In [ ]:
for epoch in range(Nepoch):
  ######################### data generation
  # randomly select a subset of rows equal to batch size
  skip=sorted(random.sample(range(nfiles),nfiles-batchsize))
  train_df=pd.read_csv(train_file,skiprows=skip)

  train_batch=train_df.to_numpy()
  filenames=train_batch[:,0]
  filelabels=train_batch[:,1]

  X=torch.zeros((batchsize,3,M,K)).to(mydevice)
  labels=torch.zeros((batchsize,1)).to(mydevice)
  nbatch=0
  for fname,flab in zip(filenames,filelabels):
    labels[nbatch,0]=float(flab)
    fullname=train_dir+os.sep+fname[0]+os.sep+fname[1]+os.sep+fname[2]+os.sep+fname+'.npy'
    # preprocessed file
    cachedfile=cache_dir+os.sep+fname[0]+os.sep+fname[1]+os.sep+fname[2]+os.sep+fname+'.npy'
    fulldir=cache_dir+os.sep+fname[0]+os.sep+fname[1]+os.sep+fname[2]
    # check if preprocessed file exists
    if not os.path.exists(fulldir):
      os.makedirs(fulldir)
    if not os.path.isfile(cachedfile):
      # preprocess file
      E=klt_bam(fullname,K,M+K)
      # create file
      np.save(cachedfile,E)
    else:
      # now we have a file that is preprocessed
      # size MxKx3
      E=np.load(cachedfile)

    # save as image
    #tv.utils.save_image(torch.from_numpy(E),'Img_'+fname+'.png',normalize=True)
    X[nbatch]=torch.from_numpy(E).to(mydevice)
    nbatch+=1


  ############################# training phase
  def closure():
    if torch.is_grad_enabled():
      optimizer.zero_grad()
    y=net(X)
    #print(y-labels)
    loss=criterion(y,labels)
    if loss.requires_grad:
       loss.backward()
       print('%d loss %f'%(epoch,loss.data.item()))
    return loss


  optimizer.step(closure)


# save model (and other extra items)
if save_model:
   torch.save({
            'model_state_dict':net.state_dict(),
           },'./net.model')
